In [1]:
import numpy as np
# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [65]:
def f_per_particle(m):
    classifier = GaussianNB()
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() + np.count_nonzero(m)
#     j=np.count_nonzero(m)

    return j

In [66]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

In [80]:
def _compute_position(swarm):
        """Update the position matrix of the swarm
        This computes the next position in a binary swarm. It compares the
        sigmoid output of the velocity-matrix and compares it with a randomly
        generated matrix.
        Parameters
        ----------
        swarm: pyswarms.backend.swarms.Swarm
            a Swarm class
        """
        
        temp=np.random.random_sample(size=(swarm.n_particles,swarm.dimensions)) < swarm.velocity
        postion=np.empty((my_swarm.n_particles, my_swarm.dimensions))
        postion1=swarm.position
        
        for j in range(swarm.n_particles):
            for i in range(len(temp[0])):
                if (temp[j][i] is True ):
                    postion[j][i]=1-postion1[j][i]
                else:
                    postion[j][i]=postion1[j][i]
                
        
        return postion

In [81]:
def compute_velocity(my_swarm, stickness, bounds=None):
    """Update the velocity matrix
    This method updates the velocity matrix using the best and current
    positions of the swarm. The velocity matrix is computed using the
    cognitive and social terms of the swarm. The velocity is handled
    by a :code:`VelocityHandler`.
    A sample usage can be seen with the following:
    .. code-block :: python
        import pyswarms.backend as P
        from pyswarms.swarms.backend import Swarm, VelocityHandler
        my_swarm = P.create_swarm(n_particles, dimensions)
        my_vh = VelocityHandler(strategy="invert")
        for i in range(iters):
            # Inside the for-loop
            my_swarm.velocity = compute_velocity(my_swarm, clamp, my_vh, bounds)
    Parameters
    ----------
    swarm : pyswarms.backend.swarms.Swarm
        a Swarm instance
    clamp : tuple of floats, optional
        a tuple of size 2 where the first entry is the minimum velocity
        and the second entry is the maximum velocity. It
        sets the limits for velocity clamping.
    vh : pyswarms.backend.handlers.VelocityHandler
        a VelocityHandler object with a specified handling strategy.
        For further information see :mod:`pyswarms.backend.handlers`.
    bounds : tuple of numpy.ndarray or list, optional
        a tuple of size 2 where the first entry is the minimum bound while
        the second entry is the maximum bound. Each array must be of shape
        :code:`(dimensions,)`.
    Returns
    -------
    numpy.ndarray
        Updated velocity matrix
    """
    try:
        # Prepare parameters
        swarm_size = my_swarm.position.shape
        alpha = my_swarm.options["c1"]
        gb=my_swarm.best_pos
        Pb= my_swarm.pbest_pos
        updated_velocity= np.empty((my_swarm.n_particles, my_swarm.dimensions))
        for p in range(my_swarm.n_particles):
            p_postion=my_swarm.position[p]
            pb=Pb[p]
            
            for i in range(len(p_postion)):
                
                
                if (p_postion[i] == pb[i] == gb[i]):
                    updated_velocity[p][i]=Is*(1-stickness[p][i])
                elif(p_postion[i] == pb[i] != gb[i]):
                    updated_velocity[p][i]=Is*(1-stickness[p][i] - (1/(alpha+1))) +(1/(alpha+1))
                elif(p_postion[i] != pb[i] and p_postion[i]== gb[i]):
                    updated_velocity[p][i]=Is*(1-stickness[p][i] - (alpha/(alpha+1))) +(alpha/(alpha+1))
                elif(p_postion[i] != pb[i] and pb[i]== gb[i]):
                    updated_velocity[p][i]= 1-Is*stickness[p][i]
        

    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    except KeyError:
        rep.logger.exception("Missing keyword in swarm.options")
        raise
    else:
        return updated_velocity

In [82]:
def compute_stickness(stickness, old_position):
    
    new_stickness=np.empty((my_swarm.n_particles, my_swarm.dimensions))
    new_position=my_swarm.position
    
    for j in range(my_swarm.n_particles):
        
        for i in range(len(stickness[j])):
            if(new_position[j][i] == old_position[j][i]):
                notchanged[j][i]+=1
                if(notchanged[j][i] < ustkS):
                    val=stickness[j][i] - (1/ustkS)
                    if(val > 0):
                        new_stickness[j][i]=val
                    else:
                        new_stickness[j][i]=0
                else:
                    new_stickness[j][i]=0   
            else:
                notchanged[j][i]=0
                new_stickness[j][i]=1
    return new_stickness

In [85]:
from sklearn.naive_bayes import GaussianNB

In [86]:
from sklearn.model_selection import train_test_split
df=pd.read_csv('D:\\SDP\\Dataset Dr.Sultan\\ant.csv')
X=np.array(df.drop('bug', axis=1))
y=np.array(df.bug)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [87]:
my_topology = Star() # The Topology Class
my_options = {'c1': 0.6} # arbitrarily set
my_swarm = P.create_swarm(n_particles=50,
                          dimensions=X.shape[1], options=my_options, discrete=True, binary=True) # The Swarm Class

print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))

The following are the attributes of our swarm: dict_keys(['position', 'velocity', 'n_particles', 'dimensions', 'options', 'pbest_pos', 'best_pos', 'pbest_cost', 'best_cost', 'current_cost'])


In [84]:
iterations=100

stickness=np.random.random_sample(size=(my_swarm.n_particles,my_swarm.dimensions))
notchanged=np.zeros((my_swarm.n_particles,my_swarm.dimensions))


ustkSL= 1*(iterations/100)
ustkSU= 8*(iterations/100)
Isu=0
Isl=10/my_swarm.dimensions

for i in range(iterations):
    my_swarm.current_cost = f(my_swarm.position) # Compute current cost
    my_swarm.pbest_cost = f(my_swarm.pbest_pos)  # Compute personal best pos
    my_swarm.pbest_pos, my_swarm.pbest_cost = P.compute_pbest(my_swarm) # Update and store
    
    Is=Isu-(i/iterations) *(Isu-Isl)
    ustkS = ustkSL + (i/iterations) *(ustkSU- ustkSL)
         
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)
    if i%20==0:
            print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))

    my_swarm.velocity = compute_velocity(my_swarm, stickness)
    old_position= my_swarm.position
    my_swarm.position = _compute_position(my_swarm)
    stickness= compute_stickness(stickness,old_position)
    

Iteration: 1 | my_swarm.best_cost: 2.1911
Iteration: 21 | my_swarm.best_cost: 2.1911
Iteration: 41 | my_swarm.best_cost: 2.1911
Iteration: 61 | my_swarm.best_cost: 2.1911
Iteration: 81 | my_swarm.best_cost: 2.1911


In [ ]:
classifier = GaussianNB()
X_selected_features = X_train[:,my_swarm.best_pos==1]
X_selected_features_test= X_test[:,my_swarm.best_pos==1]
classifier.fit(X_selected_features, y_train)
subset_performance = (classifier.predict(X_selected_features) == y_test).mean()

In [61]:
subset_performance

0.8187919463087249

In [64]:
fr = open("Results/with corssover/NB.txt", "a")
fr.write('NB\nparameters: '+str(my_options)+"\n")
fr.write("NB-ant dataset:\ncost= "+str(my_swarm.best_cost)+"\nNumber of selected features:"+str(len([i for i in my_swarm.best_pos if i==1]))+"\nSelected features:"+str(my_swarm.best_pos))
fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
fr.close()